In [1]:
# libraries required, please install pandas
import pandas as pd
from unidecode import unidecode
import re
import datetime
from datetime import date
import json
import numpy as np
import os
import time
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [3]:
base_path="/Users/sijiawu/Work/Thesis/Data/"

In [2]:
j_data=pd.read_excel('./031_recon/jstor_data.xlsx')

post_1970s=pd.read_excel('./031_recon/refs_post_1970.xlsx')
cits_post_1970s=pd.read_excel('./031_recon/network_cit_post.xlsx')

post_1970s_v2=pd.read_excel('./031_recon/refs_post_1970_v2.xlsx')
cits_post_1970s_v2=pd.read_excel('./031_recon/network_cit_post_v2.xlsx')
post_1970s_v2['btch']=post_1970s_v2['btch']+'_v2'
post_1970s_v2['f_key']=post_1970s_v2['f_key']+'_v2'

pre_1970s=pd.read_excel('./031_recon/refs_pre_1970.xlsx')
cits_pre_1970s=pd.read_excel('./031_recon/network_cit_pre.xlsx')
pre_1970s=pre_1970s.rename(columns={'id': "ref_ord"})


In [3]:
j_data=j_data.rename(columns={"ID":"id_o"})

In [4]:
pre_1970s['type'].value_counts()

3    15433
2    13810
4    12518
1     6705
8     3563
7      993
6      435
5       23
Name: type, dtype: int64

In [5]:
pre_sub=pre_1970s[['ref_ord', 'id_o','year_o', 'type',
       'author', 'title', 'journal', 'year', 'volume', 'issue', 'pages',
       'chapter_title', 'location', 'publisher', 'text_full', 'jstor',
       'journal_proc', 'year_proc', 'year_latest',
       'volume_proc', 'issue_proc', 'title_proc', 'match_url', 'outcome',
       'match_id', 'f_key']]


In [6]:
post_sub=post_1970s[['id_o', 'year_o','ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
        'f_key', 'year_proc',
       'year_latest', 'outcome', 'match_url', 'match_id']]


In [7]:
post_sub_v2=post_1970s_v2[['id_o', 'year_o','ref_ord', 'authors', 'year', 'title',
       'month', 'publisher', 'pages', 'full_reference', 'status', 'alt_j',
        'f_key', 'year_proc',
       'year_latest', 'outcome', 'match_url', 'match_id']]

In [8]:
all_refs=pd.concat([pre_sub,post_sub,post_sub_v2 ],axis=0).reset_index(drop=True)

In [9]:
all_refs["match_id"]=all_refs["match_url"].str.split('/').str[-1]

In [10]:
print(all_refs.columns)
print(all_refs.shape)

Index(['ref_ord', 'id_o', 'year_o', 'type', 'author', 'title', 'journal',
       'year', 'volume', 'issue', 'pages', 'chapter_title', 'location',
       'publisher', 'text_full', 'jstor', 'journal_proc', 'year_proc',
       'year_latest', 'volume_proc', 'issue_proc', 'title_proc', 'match_url',
       'outcome', 'match_id', 'f_key', 'authors', 'month', 'full_reference',
       'status', 'alt_j'],
      dtype='object')
(534347, 31)


In [11]:
cit_network=all_refs[(all_refs['match_id'].isna()==False)&(all_refs['id_o']!=all_refs['match_id'])].reset_index(drop=True).drop_duplicates(subset= ['id_o', 'match_id'], keep='first')[['id_o','match_id', 'f_key']]

In [12]:
relevant=all_refs[all_refs['f_key'].isin(list(cit_network['f_key']))]

In [15]:
j_data['id_o']=j_data['id_o'].astype(str)
cit_network['match_id']=cit_network['match_id'].astype(str)
cit_network['id_o']=cit_network['id_o'].astype(str)

version=str(datetime.datetime.now()).replace(' ','_')
j_data.drop_duplicates().to_excel('./031_recon/j_data_final_'+version+'.xlsx', index=False)
cit_network.drop_duplicates(subset=['id_o','match_id'],keep='first').to_excel('./031_recon/network_cit_1940_2020_'+version+'.xlsx', index=False)
all_refs.to_excel('./031_recon/refs_1940_2020_'+version+'.xlsx', index=False)
relevant.to_excel('./031_recon/refs_1940_2020_top5_'+version+'.xlsx', index=False)